# Clara Viz Video Recording
This notebook shows how to record a video of the animated rendering of volume data.

## Define the data
First the data to be rendered needs to be defined. Clara Viz provides a support class called `DataDefinition` which supports loading medical data formats and serves as a container for the data including orientation and for the settings like lights and transfer functions.

In [ ]:
from clara.viz.core import DataDefinition

data_definition = DataDefinition()
data_definition.append('data/syn3193805/img0066.nii.gz', 'DXYZ')
data_definition.append('data/syn3193805/label0066.nii.gz', 'MXYZ')
data_definition.load_settings('data/syn3193805/settings.json')

## Create the renderer using the data definition

In [ ]:
from clara.viz.core import Renderer
renderer = Renderer(data_definition)

## Set the video stream up

In [ ]:
# open the video file
file = open('./video.mp4', 'wb')

# create a video stream
video_stream = renderer.create_video_stream(lambda data, new_stream: file.write(bytes(data)))

# configure the video stream
width = 1024
height = 768
frame_rate = 30.0

# use the 'Kush Gauge' to estimate the bit rate
# (1 - low, 2 - medium, 4 - high)
motion_factor = 1
bit_rate = int(width * height * frame_rate * motion_factor * 0.07)
video_stream.configure(width, height, frame_rate, bit_rate)

## Start recording and animate the camera

In [ ]:
import math
import time

# get the settings and find the camera
settings = renderer.get_settings()
camera = [x for x in settings['Cameras'] if x['name'] == 'Cinematic'][0]

# start the video
video_stream.play()

# rotatet the camera
for step in range(0, 360):
    camera['eye']['x'] = math.cos(math.radians(step))
    camera['eye']['z'] = math.sin(math.radians(step))
    renderer.merge_settings(settings)
    time.sleep(1/frame_rate)

# stop the stream
video_stream.stop()

# wait for all frames to arrive
time.sleep(1)

# close the file
file.close()

## View the video file

In [ ]:
import IPython
IPython.display.Video('./video.mp4')